# HopBox Inference #
This notebook is used to run inference on a trained FCN semantic segmentation model trained to segment Hop Cones in an image.

The notebook loads the model and image data paths, color corrects the images, reads the info stored in QR code, and then runs inference on the images.

The notebook outputs the segmentaion results to a user specified directory.

The notebook also outputs a `.csv` file with the results of the inference run. The .csv file contains the following information:

- Image Name
- QR code informaton
- Number of hop cones in the image
- A few region properties of each hop cone (area, centroid, bounding box, etc.)
- The median, standard deviation, and average RGB values of each hop cone


Details on the Image data used, model training, color correction, and data analysis can be found in our [paper](https://....).


**Note:** This is intended to be run on Google colab, but can be run on a local machine or elsewhere with little modification and the appropriate dependencies installed.

# 1. Mount Google Drive
This section mounts the Google Drive to the notebook. This is necessary to access the data and model files stored on Google Drive. 

Make sure to change the path to the directory where the data and model files are stored.

It also lists the files in the working directory.

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
# list working directory here (this is the path to the folder with this notebook, the hopbox_functions.py file, the ref_babel_avg_srgb_d50.csv file and the folder containing the images you want to analyze):
%cd "/content/drive/MyDrive/HopBox_Publication/HopBox_colab/"
!pwd
!ls

Mounted at /content/drive/
/content/drive/.shortcut-targets-by-id/1_Hc8bE9_tBvnI6EohgtsHKpmJp6iE0iR/HopBox_Publication/HopBox_colab
/content/drive/.shortcut-targets-by-id/1_Hc8bE9_tBvnI6EohgtsHKpmJp6iE0iR/HopBox_Publication/HopBox_colab
analyzed_images      Hop_images     ReadMe_files
hopbox_functions.py  hoptrial1.pth  ref_babel_avg_srgb_d50.csv
HopBox_Infer.ipynb   __pycache__


# 2. Install Dependencies
This section installs the dependencies needed to run the notebook including:
- Cuda version of `PyTorch` and `TorchVision`, and `TorchAudio` : The libraries used to load the model and run inference
- `Pyzbar` : A toolbox used to read the QR code in images
- `libzbar0` : This is only necessary if running on linux based machine eg Colab. It is used by Pyzbar to read the QR code in images

Also check the assigned GPU. This is necessary to make sure the GPU is being used to run inference.

In [ ]:
# install basic dependencies
!pip install torch==1.9.0+cu111 torchvision==0.10.0+cu111 torchaudio==0.9.0 -f https://download.pytorch.org/whl/torch_stable.html
!apt install libzbar0 # if linux based machine
!pip install pyzbar

# Check current/allocated cuda GPU
!nvidia-smi

# 3. Load dependencies
This section loads the dependencies needed to run the notebook.
More functions are defined in the `hopbox_functions.py` file. Be sure to upload this file to the working directory.

In [ ]:
# Load more dependencies
from __future__ import print_function
from hopbox_functions import run_inference
import sys
import numpy as np
import os


sys.path.append(os.path.join(sys.path[0])) # add current directory to path


# 4. Load the Model and Data Paths, create output directory
This section loads the model and data paths, and creates the output directory.

In [ ]:
# Put the path of the folder with your images here
Base_folder = "/content/drive/MyDrive/HopBox_Publication/HopBox_colab/"
inference_dir = Base_folder + "Hop_images/"

filess = os.listdir(inference_dir)
print(str(len(filess)) + " image files found")
print(filess)

Save_Images = 1 # 1 for saving images, and 0 for not saving images

model_dir = Base_folder + "hoptrial1.pth"

output_directory = Base_folder + "analyzed_images/"

if Save_Images == 1:
  mask_dir = output_directory + "/Masks/"
  label_dir = output_directory + "/Mask_labels/"
  corrected_dir = output_directory + "/Corrected_imgs/"

  try:
    os.makedirs(mask_dir)
    os.makedirs(label_dir)
    os.makedirs(corrected_dir)
  except:
    print("'" + output_directory+ "' already exists")
    
directories = [inference_dir,
               mask_dir,
               label_dir,
               corrected_dir,
               model_dir,
               output_directory
]

# 5. Run Inference
This section loops through the images in the data directory, reads the QR code, color corrects the image, runs inference, and saves the results to the output directory.

It also saves the results described above to a `.csv` file.

In [ ]:
scale = 1; # should be based image size and network input size

# Reference Data 
def load_csv_chart(f):
    '''Input CSV's shape is (24, 3), with sRGB reference values
    '''
    data = np.loadtxt(f,delimiter=",")
    assert data.shape ==(24, 3)
    return data
ref = load_csv_chart(Base_folder + 'ref_babel_avg_srgb_d50.csv') # loads reference RGB color values

run_inference(ref=ref, directories=directories, filess=filess, scale=scale, Save_Images=Save_Images) # run inference on images in directory